## Load dataset


In [48]:
import csv
import numpy as np
import pandas as pd
from IPython.display import Image
from IPython.display import SVG
from graphviz import Source
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier


In [49]:
data  = pd.read_csv('results7parsed.csv')

# sort by best CA
avg_ca = data.sort_values('Average CA', ascending=False)
avg_rfw = data.sort_values('RFW CA', ascending=False)
avg_dt = data.sort_values('DT CA', ascending=False)

# get best 5 average features
with open('CA Features.csv', 'w')as f :
    f.write('Avg')
    f.write(avg_ca[:5].to_csv())
    
# get best 5 random forest features
with open('CA Features.csv', 'a')as f :
    f.write('Rfw')
    f.write(avg_rfw[:5].to_csv())
    
# get 5 best decision tree features 
with open('CA Features.csv', 'a')as f :
    f.write('Dtree')
    f.write(avg_dt[:5].to_csv())
    

Now, let us take the most interesting features from the csv and write a parser to analyze the feature importances.

In [50]:

s ='''nof_OH, nof_NH2, nof_SO3H, C_R0, nof_HBA, PSA/Area, molLogS
nof_OH, posCharge/Volume, C_R0, nof_HBA, PSA/Area, molLogS, molLogP
nof_OH, nof_NH2, nof_SO3H, C_sp3, C_R2, nof_HBA, PSA/Area
nof_OH, nof_NH2, nof_PO4, C_R0, nof_posCharge, nof_HBA, PSA/Area
nof_OH, nof_NH2, nof_SO3H, negCharge/Volume, nof_HBA, PSA/Area, molLogS
nof_OH, nof_NH2, nof_SO3H, C_sp3, C_R2, nof_negCharge, PSA/Area
nof_SO3H, posCharge/Volume, C_R1, nof_posCharge, nof_HBA, PSA/Area, molLogS
nof_OH, nof_SO3H, nof_negCharge, nof_posCharge, PSA/Area, molPSA, molLogP
nof_OH, nof_NH2, nof_SO3H, C_sp3, C_R2, nof_posCharge, PSA/Area
nof_OH, nof_NH2, nof_SO3H, C_R0, nof_posCharge, nof_HBA, molPSA
nof_OH, nof_NH2, nof_SH, C_R0, nof_posCharge, nof_HBA, molPSA
nof_SO3H, C_sp3, C_R0, C_R2, nof_HBA, molLogS, molLogP
PSA/Area, nof_Rings, Complexity, nof_SO3H, nof_OH, nof_Chirals, C_R0'''

features =  [sorted([x]) for x in s.split('\n')]
print(features)


[['nof_OH, nof_NH2, nof_SO3H, C_R0, nof_HBA, PSA/Area, molLogS'], ['nof_OH, posCharge/Volume, C_R0, nof_HBA, PSA/Area, molLogS, molLogP'], ['nof_OH, nof_NH2, nof_SO3H, C_sp3, C_R2, nof_HBA, PSA/Area'], ['nof_OH, nof_NH2, nof_PO4, C_R0, nof_posCharge, nof_HBA, PSA/Area'], ['nof_OH, nof_NH2, nof_SO3H, negCharge/Volume, nof_HBA, PSA/Area, molLogS'], ['nof_OH, nof_NH2, nof_SO3H, C_sp3, C_R2, nof_negCharge, PSA/Area'], ['nof_SO3H, posCharge/Volume, C_R1, nof_posCharge, nof_HBA, PSA/Area, molLogS'], ['nof_OH, nof_SO3H, nof_negCharge, nof_posCharge, PSA/Area, molPSA, molLogP'], ['nof_OH, nof_NH2, nof_SO3H, C_sp3, C_R2, nof_posCharge, PSA/Area'], ['nof_OH, nof_NH2, nof_SO3H, C_R0, nof_posCharge, nof_HBA, molPSA'], ['nof_OH, nof_NH2, nof_SH, C_R0, nof_posCharge, nof_HBA, molPSA'], ['nof_SO3H, C_sp3, C_R0, C_R2, nof_HBA, molLogS, molLogP'], ['PSA/Area, nof_Rings, Complexity, nof_SO3H, nof_OH, nof_Chirals, C_R0']]


## Project Settings

Specified here are the paths for the data and the features to run over in the list of best features.
Each entry in the list is a list containing one single string of the features to try, comma seperated. In this way it is easy to write a script to 
add entries to try very easily. 

In [51]:

##### set hyperparams
NUM_ITER = 10 # number of times to run 10foldxval to get a statistical degree of confidence

''' HYPERPARAMS FOR DECISION TREE
 
 These parameters implement a rudimentary pruning algorithm, would ideally like to use AB pruning'''
enable_pruning = True
# maximum depth of dtree
max_depth = 5
# how many samples your need atleast, at a LEAF node
min_samples = 3

##### set parameters
path_train_data = 'train.csv'
path_test_data = 'test.csv'
path_all_data = 'Dataset Correlated Removed.csv'

# set features here

best_features = features

best_features = [list(map(str.strip, x[0].split(','))) for x in best_features]

k = len(best_features)


## Load Dataset

This code loads dataset into the variables below and converts the labels to categorical 0, 1 pairs.

In [52]:
# load dataset
all_data = pd.DataFrame(pd.read_csv(path_all_data))
all_labels = all_data['SLC'].astype('category').cat.codes
# drop labels
all_data.drop('SLC', axis=1, inplace=True)

train_data = pd.DataFrame(pd.read_csv(path_train_data))
train_labels = train_data['SLC'].astype('category').cat.codes
# drop labels

train_data.drop('SLC', axis=1, inplace=True)

test_data = pd.DataFrame(pd.read_csv(path_test_data))
test_labels = test_data['SLC'].astype('category').cat.codes
# drop labels
test_data.drop('SLC', axis=1, inplace=True)


## AUC and Classification Accuracy - Decision Tree

The code below will find the classification accuracy using 10-fold cross-validation using stratified sampling to help class imbalance. The AUC on the test split is also found.

In [53]:
# visualize decision tree for input features

d_trees = []


# find CA - uses 10-fold cross validation 
# with stratified sampling to help with class imbalance
# and simple average over subsets
dt_cas = []

# maintain list of cas over a period
dt_ca_matrix = []

# run the thing NUM_ITER times
for _ in range(NUM_ITER):
    for i in range(k):
        aucs = []
        # make fold
        skf = StratifiedKFold(n_splits=10, shuffle=True)
        for trx, tex in skf.split(all_data, all_labels):
            # strip data to required features
            subset_data = all_data.filter(best_features[i], axis=1)
            
            # find auc
            dtree = DecisionTreeClassifier(presort=True, max_depth=max_depth, min_samples_leaf=min_samples)
            dtree.fit(subset_data.iloc[trx, :], all_labels.iloc[trx])        
            pred = dtree.predict(subset_data.iloc[tex, :])
            labels = all_labels.iloc[tex]
            
            acc = roc_auc_score(labels, pred)
            # record auc to average later
            aucs.append(acc)
        
        dt_cas.append(np.mean(aucs))
        
    dt_ca_matrix.append(list(dt_cas))
    dt_cas.clear()


dt_ca_matrix = np.array(dt_ca_matrix)
dt_cas = dt_ca_matrix.mean(axis=0)
dt_cas_std = dt_ca_matrix.std(axis=0)

# find AUC 
dt_aucs = []

# run k-fold validation
for i in range(k):
    subset_test_data = test_data.filter(best_features[i], axis=1)
    subset_train_data = train_data.filter(best_features[i], axis=1)
    
    clf = DecisionTreeClassifier(presort=True, max_depth=max_depth, min_samples_leaf=min_samples)
    clf.fit(subset_train_data, train_labels)
    d_trees.append(clf)
    
    # make its predictions on test data
    pred = d_trees[i].predict(subset_test_data)
    
    # find auc scores
    auc = roc_auc_score(test_labels, pred)
    
    # record the scores
    dt_aucs.append(auc)


print('Decision Tree Results:')
print('   \tAUC\tAcc (%dit) (mean ± std)\t\tFeatures' % NUM_ITER)
for i, f in enumerate(zip(dt_aucs, dt_cas, dt_cas_std)):
    print('%1d' %i,'\t%05.3f\t%05.3f ± %05.03f\t' % tuple(f) + ', '.join((best_features[i])))

    

Decision Tree Results:
   	AUC	Acc (10it) (mean ± std)		Features
0 	0.714	0.675 ± 0.026	nof_OH, nof_NH2, nof_SO3H, C_R0, nof_HBA, PSA/Area, molLogS
1 	0.714	0.703 ± 0.021	nof_OH, posCharge/Volume, C_R0, nof_HBA, PSA/Area, molLogS, molLogP
2 	0.786	0.722 ± 0.009	nof_OH, nof_NH2, nof_SO3H, C_sp3, C_R2, nof_HBA, PSA/Area
3 	0.738	0.695 ± 0.023	nof_OH, nof_NH2, nof_PO4, C_R0, nof_posCharge, nof_HBA, PSA/Area
4 	0.726	0.707 ± 0.016	nof_OH, nof_NH2, nof_SO3H, negCharge/Volume, nof_HBA, PSA/Area, molLogS
5 	0.786	0.724 ± 0.020	nof_OH, nof_NH2, nof_SO3H, C_sp3, C_R2, nof_negCharge, PSA/Area
6 	0.738	0.664 ± 0.028	nof_SO3H, posCharge/Volume, C_R1, nof_posCharge, nof_HBA, PSA/Area, molLogS
7 	0.774	0.675 ± 0.022	nof_OH, nof_SO3H, nof_negCharge, nof_posCharge, PSA/Area, molPSA, molLogP
8 	0.786	0.725 ± 0.023	nof_OH, nof_NH2, nof_SO3H, C_sp3, C_R2, nof_posCharge, PSA/Area
9 	0.798	0.707 ± 0.018	nof_OH, nof_NH2, nof_SO3H, C_R0, nof_posCharge, nof_HBA, molPSA
10 	0.798	0.722 ± 0.024	nof_OH, nof_NH2,

## AUC and Classification Accuracy - Random Forest Walk

The code below will find the classification accuracy using 10-fold cross-validation using stratified sampling to help class imbalance. The AUC on the test split is also found.

In [54]:
# visualize random forest features
rfws = []


# find CA - uses 10-fold cross validation 
# with stratified sampling to help with class imbalance
# and simple average over subsets
rfw_cas = []

# maintain list of cas over a period
rfw_ca_matrix = []

# run the thing NUM_ITER times
for _ in range(NUM_ITER):
    for i in range(k):
        aucs = []
        # make fold
        skf = StratifiedKFold(n_splits=10, shuffle=True)
        for trx, tex in skf.split(all_data, all_labels):
            # strip data to required features
            subset_data = all_data.filter(best_features[i], axis=1)
            
            # find auc
            rfwtree = RandomForestClassifier(n_estimators=100)
            rfwtree.fit(subset_data.iloc[trx, :], all_labels.iloc[trx])
            pred = rfwtree.predict(subset_data.iloc[tex, :])
            labels = all_labels.iloc[tex]
            
            acc = roc_auc_score(labels, pred)
            # record auc to average later
            aucs.append(acc)
        
        rfw_cas.append(np.mean(aucs))
    rfw_ca_matrix.append(list(rfw_cas))
    rfw_cas.clear()


rfw_ca_matrix = np.array(rfw_ca_matrix)
rfw_cas = rfw_ca_matrix.mean(axis=0)
rfw_cas_std = rfw_ca_matrix.std(axis=0)

# find AUC 
rfw_aucs = []
for i in range(k):
    subset_test_data = test_data.filter(best_features[i], axis=1)
    subset_train_data = train_data.filter(best_features[i], axis=1)
    
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(subset_train_data, train_labels)
    rfws.append(clf)
    
    # make its predictions on test data
    pred = rfws[i].predict(subset_test_data)
    
    # find auc scores
    auc = roc_auc_score(test_labels, pred)
    
    # record the scores
    rfw_aucs.append(auc)


print('Random Forest Results:')
print('   \tAUC\tAcc (%dit) (mean ± std)\t\tFeatures' % NUM_ITER)
for i, f in enumerate(zip(dt_aucs, rfw_cas, rfw_cas_std)):
    print('%1d' % i, '\t%05.3f\t%05.3f ± %05.03f\t' % tuple(f) + ', '.join(sorted(best_features[i])))


Random Forest Results:
   	AUC	Acc (10it) (mean ± std)		Features
0 	0.714	0.786 ± 0.014	C_R0, PSA/Area, molLogS, nof_HBA, nof_NH2, nof_OH, nof_SO3H
1 	0.714	0.762 ± 0.012	C_R0, PSA/Area, molLogP, molLogS, nof_HBA, nof_OH, posCharge/Volume
2 	0.786	0.765 ± 0.012	C_R2, C_sp3, PSA/Area, nof_HBA, nof_NH2, nof_OH, nof_SO3H
3 	0.738	0.794 ± 0.011	C_R0, PSA/Area, nof_HBA, nof_NH2, nof_OH, nof_PO4, nof_posCharge
4 	0.726	0.757 ± 0.015	PSA/Area, molLogS, negCharge/Volume, nof_HBA, nof_NH2, nof_OH, nof_SO3H
5 	0.786	0.761 ± 0.023	C_R2, C_sp3, PSA/Area, nof_NH2, nof_OH, nof_SO3H, nof_negCharge
6 	0.738	0.758 ± 0.014	C_R1, PSA/Area, molLogS, nof_HBA, nof_SO3H, nof_posCharge, posCharge/Volume
7 	0.774	0.725 ± 0.010	PSA/Area, molLogP, molPSA, nof_OH, nof_SO3H, nof_negCharge, nof_posCharge
8 	0.786	0.742 ± 0.012	C_R2, C_sp3, PSA/Area, nof_NH2, nof_OH, nof_SO3H, nof_posCharge
9 	0.798	0.772 ± 0.017	C_R0, molPSA, nof_HBA, nof_NH2, nof_OH, nof_SO3H, nof_posCharge
10 	0.798	0.778 ± 0.013	C_R0, molPSA, no

 
## Tabulate Results


In [65]:
to_write = [['AUC DT','AUC RFW','Acc DT', 'Std Acc DT' ,'Acc RFW', 'Std Acc RFW', 'Features']]

print('\tAUC(DT)\tAUC(RFW)Acc(DT %dit, x± std)\tAcc(RFW %dit, x±std)\tFeatures' % (NUM_ITER, 
                                                                                                     NUM_ITER))
for i, f in enumerate(zip(dt_aucs, rfw_aucs, dt_cas, dt_cas_std, rfw_cas, rfw_cas_std)):
    print('%2d' % i, '\t%05.3f\t%05.3f\t%05.3f±%05.03f\t%05.3f±%04.03f\t' % tuple(f) + ', '.join((best_features[i])))
    to_write.append(list(f) + [', '.join((best_features[i]))])
    
# write results to a csv file
output = csv.writer(open('Final Result.csv', 'w', newline=''))
output.writerows(to_write)


	AUC(DT)	AUC(RFW)Acc(DT 10it, x± std)	Acc(RFW 10it, x±std)	Features
 0 	0.714	0.905	0.675±0.026	0.786±0.014	nof_OH, nof_NH2, nof_SO3H, C_R0, nof_HBA, PSA/Area, molLogS
 1 	0.714	0.905	0.703±0.021	0.762±0.012	nof_OH, posCharge/Volume, C_R0, nof_HBA, PSA/Area, molLogS, molLogP
 2 	0.786	0.869	0.722±0.009	0.765±0.012	nof_OH, nof_NH2, nof_SO3H, C_sp3, C_R2, nof_HBA, PSA/Area
 3 	0.738	0.881	0.695±0.023	0.794±0.011	nof_OH, nof_NH2, nof_PO4, C_R0, nof_posCharge, nof_HBA, PSA/Area
 4 	0.726	0.905	0.707±0.016	0.757±0.015	nof_OH, nof_NH2, nof_SO3H, negCharge/Volume, nof_HBA, PSA/Area, molLogS
 5 	0.786	0.881	0.724±0.020	0.761±0.023	nof_OH, nof_NH2, nof_SO3H, C_sp3, C_R2, nof_negCharge, PSA/Area
 6 	0.738	0.893	0.664±0.028	0.758±0.014	nof_SO3H, posCharge/Volume, C_R1, nof_posCharge, nof_HBA, PSA/Area, molLogS
 7 	0.774	0.833	0.675±0.022	0.725±0.010	nof_OH, nof_SO3H, nof_negCharge, nof_posCharge, PSA/Area, molPSA, molLogP
 8 	0.786	0.845	0.725±0.023	0.742±0.012	nof_OH, nof_NH2, nof_SO3H, C_sp3, C

In [56]:
print(k)
print(map(len, [dt_aucs, rfw_aucs, dt_cas, dt_cas_std, rfw_cas, rfw_cas_std]))


13


## Feature importance

The feature importances are compared below for decision trees and random forests.

In [57]:
i = 0
# visualization
for dtree in d_trees:
    if i < k:
        print('Feature importances for tree and forest (resp.) %s/%s:' % (i + 1, k))
        for e in zip(dtree.feature_importances_, rfws[i].feature_importances_, best_features[i]):
            print('\t%6f\t%6f\t%s' % e)

    else:
        print('Warning, code may be buggy')
    i += 1


Feature importances for tree and forest (resp.) 1/13:
	0.000000	0.082069	nof_OH
	0.000000	0.058452	nof_NH2
	0.000000	0.016539	nof_SO3H
	0.288639	0.197948	C_R0
	0.000000	0.129128	nof_HBA
	0.580146	0.290613	PSA/Area
	0.131215	0.225251	molLogS
Feature importances for tree and forest (resp.) 2/13:
	0.000000	0.062511	nof_OH
	0.000000	0.064157	posCharge/Volume
	0.288639	0.169290	C_R0
	0.000000	0.083282	nof_HBA
	0.580146	0.230621	PSA/Area
	0.131215	0.186561	molLogS
	0.000000	0.203579	molLogP
Feature importances for tree and forest (resp.) 3/13:
	0.000000	0.087164	nof_OH
	0.074083	0.066793	nof_NH2
	0.000000	0.023955	nof_SO3H
	0.193389	0.228668	C_sp3
	0.265219	0.084492	C_R2
	0.039843	0.146552	nof_HBA
	0.427467	0.362376	PSA/Area
Feature importances for tree and forest (resp.) 4/13:
	0.000000	0.094939	nof_OH
	0.065841	0.063909	nof_NH2
	0.000000	0.022548	nof_PO4
	0.299907	0.247311	C_R0
	0.000000	0.039122	nof_posCharge
	0.010451	0.155213	nof_HBA
	0.623800	0.376957	PSA/Area
Feature importances for t